# REDES NEURAIS ARTIFICIAIS

[Aula 3 de Deep Learning](http://bit.ly/dn-unb03) da Engenharia de Software da UnB

# Parte 1 - Pré-processamento dos Dados

### Importar as libs

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
tf.__version__

'2.4.1'

### Importar o dataset

In [3]:
dataset = pd.read_csv('Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values
print("X >>\n",X)
print("y >>\n",y)

X >>
 [[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
y >>
 [1 0 1 ... 1 1 0]


### Transformando os dados categóricos


In [5]:
# Label Encoding the "Gender" column
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])
print(X)
# One Hot Encoding the "Geography" column
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]
[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


# Dividindo o dataset em conjunto de treinamento e conjunto de teste

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling

In [7]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [8]:
np.shape(X_train)

(8000, 12)

---
# Parte 2 -Vamos construir uma ANN!


### Initializing the ANN

In [9]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [10]:
ann.add(tf.keras.layers.Dense(units=12, activation='tanh', input_shape = X_train.shape[1:]))

### Adding the second hidden layer

In [11]:
ann.add(tf.keras.layers.Dense(units=18, activation='tanh'))

### Adding the output layer

In [12]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the ANN

In [13]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

EStop = EarlyStopping(monitor="val_accuracy", patience=12)
MCP_save = ModelCheckpoint('Exercicio1.hdf5', save_best_only=True, monitor='val_loss', mode='min')

### Compiling the ANN

In [14]:
lr0 = 0.01
epochs = 100
decay_rate = lr0 / epochs

In [15]:
ann.compile(optimizer = Adam(lr=lr0, decay = decay_rate), loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the ANN on the Training set

In [16]:
ann.fit(X_train, y_train,
        validation_data = (X_test, y_test),
        batch_size = 32, epochs = epochs,
        callbacks = [EStop, MCP_save])

Epoch 1/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4753 - accuracy: 0.7755 - val_loss: 0.3739 - val_accuracy: 0.8400
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3560 - accuracy: 0.8517 - val_loss: 0.3418 - val_accuracy: 0.8640
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3501 - accuracy: 0.8572 - val_loss: 0.3376 - val_accuracy: 0.8615
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3313 - accuracy: 0.8667 - val_loss: 0.3368 - val_accuracy: 0.8595
Epoch 5/100
250/250 [==============================] - 0s 996us/step - loss: 0.3302 - accuracy: 0.8659 - val_loss: 0.3410 - val_accuracy: 0.8665
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3344 - accuracy: 0.8644 - val_loss: 0.3370 - val_accuracy: 0.8650
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3284 - accuracy: 0.8686 - val_loss: 0.3358 - val_accuracy: 0.85

# Parte 3 - Fazendo predições e avaliando o modelo

## Prevendo os resultados com o conjunto de testes

In [17]:
melhor_ann = tf.keras.models.load_model("Exercicio1.hdf5")

In [18]:
y_pred = melhor_ann.predict(X_test)
y_pred

array([[0.24334073],
       [0.24950552],
       [0.08709535],
       ...,
       [0.08503035],
       [0.18460459],
       [0.32337123]], dtype=float32)

In [19]:
y_pred = (y_pred > 0.5)
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

# Criando uma Confusion Matrix

In [20]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1506   89]
 [ 181  224]]


In [21]:
acc = (cm[0][0] + cm[1][1]) / X_test.shape[0]
print(f"A acuracia do modelo estima-se em {acc*100}%")

A acuracia do modelo estima-se em 86.5%


# FIM

> Professor Diego Dorgam  
> [@diegodorgam](https://twitter.com/diegodorgam)